In [1]:
#!pip install wikipedia
#!pip install wikipedia-api
!pip install openpyxl
!pip install pandas
!pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 9.5 MB/s eta 0:00:00


In [2]:
import requests
import pandas as pd
import datetime
import time

In [3]:
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
WIKIPEDIA_API_URL = "https://fr.wikipedia.org/w/api.php"

def get_total_article_count():
    params = {
        "action": "query",
        "format": "json",
        "meta": "siteinfo",
        "siprop": "statistics"
    }

    # ceci nous permet d'avoir le nombre total d'articles en francais dans l'element articles environ 2675254
    # https://fr.wikipedia.org/w/api.php?action=query&meta=siteinfo&siprop=statistics
    response = requests.get(WIKIPEDIA_API_URL, params=params).json()
    total_articles = response["query"]["statistics"]["articles"]

    return total_articles

# Obtenir le nombre total d'articles
total_articles = get_total_article_count()
print(f"Nombre total d'articles en français sur Wikipédia : {total_articles}")


Nombre total d'articles en français sur Wikipédia : 2680047


In [ ]:
# Paramètres de l'API
# Fonction pour récupérer tous les titres des articles de Wikipédia
def get_all_article_titles(limit=500):
    # Wikipédia limite à 500 articles par requête
    all_titles = []
    continue_param = None

    while True:
        # les deux derniers params nous permette d'extraire uniquement les articles et non toutes les pages qui incluent
        # les pages de discussions, les catégories, les fichiers, images, les pages systems, les redirections, brouillons et standbox, etc
        # qui est ce que renvoie la requête API list=allpages par défaut sans filtre
        params = {
            "action": "query",
            "format": "json",
            "list": "allpages",
            "aplimit": limit,
            "apfilterredir": "nonredirects",  # Exclure les redirections
            "apnamespace": 0  # Exclure les catégories, fichiers...
        }
        if continue_param:
            params.update(continue_param)

        response = requests.get(WIKIPEDIA_API_URL, params=params).json()
        pages = response["query"]["allpages"]
        all_titles.extend([page["title"] for page in pages])

        # Vérification s'il y a une suite
        if "continue" in response:
            continue_param = response["continue"]
        else:
            break

        print(f"{len(all_titles)} articles récupérés...")
        print(f"Le titre du dernier article est : {all_titles[-1]}")
        time.sleep(1)  # Petite pause pour éviter de surcharger l'API

    return all_titles



In [ ]:
# Récupération des titres de tous les articles
# CECI PEUT PRENDRES DES HEURES
print("Récupération des titres des articles...")
article_titles = get_all_article_titles(limit=500)

print("End of extraction...")



In [ ]:
# Convertir la liste en DataFrame
df_titles = pd.DataFrame(article_titles, columns=["Titre"])

print(df_titles)

In [ ]:
# Définir le chemin du fichier TXT pour sauvegarder dans un fichier TXT
print("Saving as ")
txt_path = "/content/drive/MyDrive/wikipedia_fr_articles_titles.txt"

# Écrire chaque titre dans une nouvelle ligne
with open(txt_path, "w", encoding="utf-8") as f:
    for title in article_titles:
        f.write(title + "\n")

print("Fichier TXT enregistré dans Google Drive : wikipedia_fr_articles_titles.txt")

In [ ]:
# Pour sauvegarder en csv qui n'impose pas de limites de ligne
df_titles.to_csv("/content/drive/MyDrive/wikipedia_fr_articles_titles.csv", index=False)

In [ ]:
#Ceci ne marche pas vu qu'on a plus d'un millions et quelque de ligne qu'excel ne peut supporter

# Monter Google Drive
drive.mount('/content/drive')

# Sauvegarder dans Google Drive
print("Saving as Excel file")
df_titles.to_excel("/content/drive/MyDrive/wikipedia_fr_articles_titles.xlsx", index=False, engine="openpyxl")
print("Fichier Excel enregistré dans Google Drive : wikipedia_fr_articles_titles.xlsx")


In [ ]:
# Vu que la liste des articles est pres de 3 millions, excel ne peut l'enregistré en une seule feuille, il nous faut donc plusieurs fichiers
batch_size = 1_000_000  # Nombre de lignes max par fichier
num_files = (len(article_titles) // batch_size) + 1  # Nombre de fichiers à créer

for i in range(num_files):
    start = i * batch_size
    end = start + batch_size
    df = pd.DataFrame(article_titles[start:end], columns=["Titre"])

    # Sauvegarde dans des fichiers nommés wikipedia_titles_part1.xlsx, part2, etc.
    file_name = f"/content/drive/MyDrive/wikipedia_fr_articles_titles_part{i+1}.xlsx"
    df.to_excel(file_name, index=False, engine="openpyxl")

    print(f"Fichier {file_name} enregistré avec {len(df)} lignes")


In [ ]:
# Sauvegarder en format SQLite (base de données) :

import sqlite3

conn = sqlite3.connect("/content/drive/MyDrive/wikipedia_fr_articles_titles.db")
df_titles.to_sql("titles", conn, if_exists="replace", index=False)
conn.close()##
 print(f"Fichier {file_name} enregistré avec {len(df_titles)} lignes")

In [ ]:
# Fonction pour récupérer le contenu d'un article à la fois
# s'est avéré tres lent, donc on laisse d'abord pour faire des extractions de 50 articles à la fois

def get_article_content_one_on_one(title):
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "explaintext": True,
        "titles": title
    }
    '''
    response = requests.get(WIKIPEDIA_API_URL, params=params).json()
    page = next(iter(response["query"]["pages"].values()))
    return page.get("extract", "")
    '''
    # On ajoute try except pour gerer l'erreur SSL qui apparaissait
    try:
        response = requests.get(WIKIPEDIA_API_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        page = next(iter(data["query"]["pages"].values()))
        return page.get("extract", "Pas de contenu")

    except requests.exceptions.SSLError:
        print(f"SSLError pour l’article : {title}")
        return "Pas de contenu (SSLError)"

    except Exception as e:
        print(f"Erreur pour l’article : {title} => {e}")
        return "Pas de contenu (Erreur)"


#print(article_titles)

In [ ]:
# Here I already have save all the title in a csv files, I just need to upload it as a list and a df
print("Début Lecture du fichier csv")
df_titles = pd.read_csv("/content/drive/MyDrive/wikipedia_fr_articles_titles.csv")
print("Fin Lecture du fichier csv")

article_titles = df_titles["Titre"].tolist()

#print("Affichage de la liste")
#print(article_titles)

print("Affichage du df")
print(df_titles)

Début Lecture du fichier csv
Fin Lecture du fichier csv
Affichage du df
                        Titre
0                          !!
1                         !!!
2                 !!! (album)
3               !Action Pact!
4        !Mediengruppe Bitnik
...                       ...
2675275                     𝼦
2675276                     𝼪
2675277                     𞟠
2675278                     🏧
2675279                     🜨

[2675280 rows x 1 columns]


In [ ]:
print("Affichage de la liste")
print(article_titles[:20])

Affichage de la liste
['!!', '!!!', '!!! (album)', '!Action Pact!', '!Mediengruppe Bitnik', '!T.O.O.H.!', '!Wowow!', '! (album)', '! (homonymie)', '"@"', '"A" Device', '"C" Device', '"Heroes"', '"Heroes" (chanson de David Bowie)', '"R" Device', '"This Is Our Punk-Rock," Thee Rusted Satellites Gather+Sing', '"Uh Uh,,,,,," feat. AI / Baby Be Mine', '"V" Device', '$10', '$100,000']


In [ ]:
print("Affichage du df")
print(df_titles)

Affichage du df
                        Titre
0                          !!
1                         !!!
2                 !!! (album)
3               !Action Pact!
4        !Mediengruppe Bitnik
...                       ...
2675275                     𝼦
2675276                     𝼪
2675277                     𞟠
2675278                     🏧
2675279                     🜨

[2675280 rows x 1 columns]


In [ ]:
# ceci n'a pas marché
# Fonction pour récupérer le contenu d'un lot d'articles (50 max par requête)
def get_articles_content(titles):
    articles = []

    # Paramètres de la requête
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": "|".join(titles),  # Requête avec les titres séparés par |
        "explaintext": True,  # Extraire le texte brut sans balises HTML
        "exlimit": "max"
    }

    # Requête API
    response = requests.get(WIKIPEDIA_API_URL, params=params).json()

    # Extraire les informations des pages récupérées
    for page in response["query"]["pages"].values():
        title = page["title"]
        content = page.get("extract", "Pas de contenu")
        link = f"https://fr.wikipedia.org/wiki/{title.replace(' ', '_')}"
        articles.append([datetime.datetime.today().strftime("%Y-%m-%d"), title, link, content])

    return articles


In [ ]:
# ceci n'a pas marché
# Rappel : article_titles contient notre liste de pres de 3 millions titres
# Extraction des articles en lots de 50 (jusqu'à 3 millions d'articles)
all_data = []  # Liste pour stocker les résultats
total_titles = len(article_titles)

print("Début de l'extraction")
# Extraction par lots de 50 titres
for i in range(0, total_titles, 50):  # Parcours la liste par groupes de 50 titres
    batch_titles = article_titles[i:i + 50]  # Récupère 50 titres à la fois
    data = get_articles_content(batch_titles)  # Récupère leur contenu
    all_data.extend(data)  # Ajoute les résultats extraits

    print(f"{i + len(batch_titles)}/{total_titles} articles extraits.")  # Affiche la progression
    time.sleep(5)  # Pause de 5 secondes pour éviter de surcharger l'API

print("Extraction terminée")
#print(all_data)

Début de l'extraction
50/2675280 articles extraits.
100/2675280 articles extraits.
150/2675280 articles extraits.
200/2675280 articles extraits.
250/2675280 articles extraits.
300/2675280 articles extraits.
350/2675280 articles extraits.
400/2675280 articles extraits.
450/2675280 articles extraits.
500/2675280 articles extraits.
550/2675280 articles extraits.
600/2675280 articles extraits.
650/2675280 articles extraits.
700/2675280 articles extraits.
750/2675280 articles extraits.
800/2675280 articles extraits.
850/2675280 articles extraits.
900/2675280 articles extraits.
950/2675280 articles extraits.
1000/2675280 articles extraits.
1050/2675280 articles extraits.
1100/2675280 articles extraits.
1150/2675280 articles extraits.
1200/2675280 articles extraits.
1250/2675280 articles extraits.
1300/2675280 articles extraits.
1350/2675280 articles extraits.
1400/2675280 articles extraits.
1450/2675280 articles extraits.
1500/2675280 articles extraits.
1550/2675280 articles extraits.
1600/2

SSLError: HTTPSConnectionPool(host='fr.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&format=json&prop=extracts&titles=Addison+Emery+Verrill%7CAddison+Forbes-Montgomery%7CAddison+Holley%7CAddison+Hutton%7CAddison+Lockley%7CAddison+Powell%7CAddison+Rae%7CAddison+Reed%7CAddison+Richards%7CAddison+Riecke%7CAddison+Russell%7CAddison+S.+McClure%7CAddison+Timlin%7CAddison+Township+%28Illinois%29%7CAddison+Webster+Moore%7CAddisoniidae%7CAdditif%7CAdditif+alimentaire%7CAdditif+pour+carburant%7CAddition%7CAddition+%28homonymie%29%7CAddition+%28restauration%29%7CAddition+Elle%7CAddition+aldolique+de+Mukaiyama%7CAddition+aux+Pens%C3%A9es+philosophiques%7CAddition+dans+l%27arithm%C3%A9tique+de+Peano%7CAddition+de+Michael%7CAddition+matricielle%7CAddition+nucl%C3%A9ophile%7CAddition+nucl%C3%A9ophile+conjugu%C3%A9e%7CAddition+oxydante%7CAddition+radicalaire%7CAddition+syn+et+anti%7CAddition+%C3%A9lectrophile%7CAdditionneur%7CAdditor+%28spiritisme%29%7CAddizione+Erc

In [ ]:
# Convertir all_data en DataFrame
df = pd.DataFrame(all_data, columns=["Date d'Extraction", "Nom de l'Article", "Lien", "Contenu"])
print(df[100:110])

    Date d'Extraction                   Nom de l'Article  \
100        2025-04-05  'Tis Time for "Torture," Princess   
101        2025-04-05        'Tis a Pity She Was a Whore   
102        2025-04-05          'Tonnage' Martin Wolfkeil   
103        2025-04-05                 'Ujayf ibn 'Anbasa   
104        2025-04-05                          'Upa 'upa   
105        2025-04-05                 'Uruq Bani Ma'arid   
106        2025-04-05                               'Zik   
107        2025-04-05                         'akava'ine   
108        2025-04-05                           'are'are   
109        2025-04-05                             'ataba   

                                                  Lien  \
100  https://fr.wikipedia.org/wiki/'Tis_Time_for_"T...   
101  https://fr.wikipedia.org/wiki/'Tis_a_Pity_She_...   
102  https://fr.wikipedia.org/wiki/'Tonnage'_Martin...   
103   https://fr.wikipedia.org/wiki/'Ujayf_ibn_'Anbasa   
104            https://fr.wikipedia.org/wiki/'Upa

In [ ]:
# Stockage des articles dans une liste
data = []

In [ ]:

#for index, title in enumerate(article_titles[:100]):  # Limite pour éviter une surcharge initiale
# Ici je mets de 15360 à 30000 parce que l'extraction avait été déjà fait jusqu'à l'index 15359 et a pris environ 5H
for index, title in enumerate(article_titles[15360:30000]):
    content = get_article_content_one_on_one(title)
    link = f"https://fr.wikipedia.org/wiki/{title.replace(' ', '_')}"
    data.append([index + 1, datetime.datetime.today().strftime("%Y-%m-%d"), title, link, content])

    print(f"{index+1}/{len(article_titles)} - {title} récupéré, contenu: {content[:25]} ...")

    # Après extraction de 50 articles, on fait une pause
    if (index + 1) % 50 == 0:
      time.sleep(5)  # Pause pour ne pas dépasser la limite de l'API


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
10361/2675280 - (18051) 1999 RU196 récupéré
10362/2675280 - (18052) 1999 RV199 récupéré
10363/2675280 - (18053) 1999 RU208 récupéré
10364/2675280 - (18054) 1999 SW7 récupéré
10365/2675280 - (18055) Fernhildebrandt récupéré
10366/2675280 - (18056) 1999 TV15 récupéré
10367/2675280 - (18057) 1999 VK10 récupéré
10368/2675280 - (18058) 1999 XY129 récupéré
10369/2675280 - (18059) Cavalieri récupéré
10370/2675280 - (1806) Derice récupéré
10371/2675280 - (18060) Zarex récupéré
10372/2675280 - (18061) 1999 XH179 récupéré
10373/2675280 - (18062) 1999 XY187 récupéré
10374/2675280 - (18063) 1999 XW211 récupéré
10375/2675280 - (18064) 1999 XY242 récupéré
10376/2675280 - (180643) Cardoen récupéré
10377/2675280 - (18065) 2000 AM41 récupéré
10378/2675280 - (18066) 2000 AR79 récupéré
10379/2675280 - (18067) 2000 AB98 récupéré
10380/2675280 - (18068) 2000 AF184 récupéré
10381/2675280 - (18069) 2000 AS199 récupéré
10382/2675280

SSLError: HTTPSConnectionPool(host='fr.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&format=json&prop=extracts&explaintext=True&titles=%2822145%29+2000+WJ17 (Caused by SSLError(SSLError(1, '[SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2580)')))

In [ ]:
# Convertir data en DataFrame
df = pd.DataFrame(data, columns=["Numéro", "Date d'Extraction", "Nom de l'Article", "Lien", "Contenu"])
print(df)

       Numéro Date d'Extraction      Nom de l'Article  \
0           1        2025-04-05                    !!   
1           2        2025-04-05                   !!!   
2           3        2025-04-05           !!! (album)   
3           4        2025-04-05         !Action Pact!   
4           5        2025-04-05  !Mediengruppe Bitnik   
...       ...               ...                   ...   
15355   15356        2025-04-05   (22140) Suzyamamoto   
15356   15357        2025-04-05     (22141) 2000 VH36   
15357   15358        2025-04-05     (22142) Loripryor   
15358   15359        2025-04-05   (22143) Cathyfowler   
15359   15360        2025-04-05   (22144) Linmichaels   

                                                    Lien  \
0                       https://fr.wikipedia.org/wiki/!!   
1                      https://fr.wikipedia.org/wiki/!!!   
2              https://fr.wikipedia.org/wiki/!!!_(album)   
3            https://fr.wikipedia.org/wiki/!Action_Pact!   
4      https://

In [ ]:
# Sauvegarde dans un fichier Excel SI LE NOMBRE D'ARTICLE EXTRAIT EST MOINS D'UN MILLION

#df = pd.DataFrame(data, columns=["Numéro", "Date d'Extraction", "Nom de l'Article", "Lien", "Contenu"])
#df.to_excel("wikipedia_articles_fr.xlsx", index=False, engine="openpyxl")

#print(f"{len(df)} articles enregistrés dans wikipedia_articles_fr.xlsx avec succès !")

# Sauvegarde dans Google Drive
df = pd.DataFrame(data, columns=["Numéro", "Date d'Extraction", "Nom de l'Article", "Lien", "Contenu"])
df.to_excel("/content/drive/MyDrive/wikipedia_fr_articles_content.xlsx", index=False, engine="openpyxl")

print(f"{len(df)} articles enregistrés dans Google Drive : wikipedia_fr_articles_content.xlsx")

In [ ]:
# Sauvegarde du df all_data (qui contient tous le contenu des articles)
# dans plusieurs fichiers excel, en csv et sqlite

# 1. Sauvegarde en plusieurs fichiers Excel

# Taille maximale d'un fichier Excel (1 million de lignes)
EXCEL_MAX_ROWS = 1_000_000

# Nombre total de fichiers nécessaires
num_files = (len(df) // EXCEL_MAX_ROWS) + 1

# Sauvegarde en plusieurs fichiers
for i in range(num_files):
    start_row = i * EXCEL_MAX_ROWS
    end_row = (i + 1) * EXCEL_MAX_ROWS
    df_chunk = df[start_row:end_row]  # Extraire le lot
    # file_name = f"wikipedia_articles_fr_contents_part_{i+1}.xlsx" #on utilise ceci, si on n'est pas sur google colab
    file_name = f"/content/drive/MyDrive/wikipedia_articles_fr_contents_part_{i+1}.xlsx"
    df_chunk.to_excel(file_name, index=False, engine="openpyxl")  # Sauvegarde Excel

    print(f"{file_name} enregistré avec ({len(df_chunk)} lignes)")

print("Tous les fichiers Excel sont sauvegardés !")


/content/drive/MyDrive/wikipedia_articles_fr_contents_part_1.xlsx enregistré avec (15360 lignes)
Tous les fichiers Excel sont sauvegardés !


In [ ]:
# 2. Sauvegarde en CSV

# Le CSV est plus léger et rapide à écrire que Excel.
# Sauvegarde de all_data en CSV
csv_filename = "/content/drive/MyDrive/wikipedia_articles_fr_contents_from_1_to_15359.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8")

print(f"Sauvegarde CSV terminée : {csv_filename}")


Sauvegarde CSV terminée : /content/drive/MyDrive/wikipedia_articles_fr_contents_from_1_to_15359.csv


In [ ]:
# 3. Sauvegarde dans une base SQLite
# SQLite permet d'enregistrer efficacement des millions de lignes et de faire des requêtes rapides.

import sqlite3

# Connexion à la base SQLite
conn = sqlite3.connect("/content/drive/MyDrive/wikipedia_articles_fr_contents.db")

# Sauvegarde dans une table nommée "articles"
df.to_sql('articles', conn, if_exists='replace', index=False)

# Fermeture de la connexion
conn.close()
'''
cursor = conn.cursor()

# Création de la table si elle n'existe pas
cursor.execute("""
CREATE TABLE IF NOT EXISTS Articles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date_extraction TEXT,
    titre TEXT,
    lien TEXT,
    contenu TEXT
)
""")

# Insertion des données dans la base SQLite
df.to_sql("Articles", conn, if_exists="append", index=False)

# Fermeture de la connexion
conn.commit()
conn.close()
'''
print("Sauvegarde dans la base SQLite terminée !")


In [ ]:
1000 % 50

0

**CONCENTRONS NOUS SUR LA TRADUCTION AVEC AZURE OPEN AI**

In [ ]:
!pip install openai

In [ ]:
from openai import AzureOpenAI

In [ ]:
# CECI EST SUPPRIMER OU REMPLACER LES VALEURS PAR AUTRES AVANT PUBLICATION

client=AzureOpenAI(
		api_key="your api key",
		api_version="your api version",
		azure_endpoint="your azure endpoint"
	)
deployment_name = "your deployment name"

In [ ]:
# Chargement du fichier
df = pd.read_csv("/content/drive/MyDrive/wikipedia_articles_fr_contents_from_1_to_15359.csv")

In [ ]:
# FONCTION DE TRADUCTION AVEC OPEN AI
def call_openai_api(prompt):
	if prompt.strip() == "":
		return ""
	try :
		response=client.chat.completions.create(
				 model=deployment_name,
				 #messages=[{"role": "user", "content": prompt}]
				 messages=[
							#{
							#		 "role": "system",
							#		 "content": "Tu es un traducteur expert du français vers le fon."
							#},
							{
									 "role": "user",
									 "content": f"traduis ce texte en fon : {prompt}"
									 #"content": f"Utilise google translate, et d'autres ressources comme : https://web.archive.org/web/20071031195535/http://langues-africaines.com:80/FONGBE/Grammaire-Plan.htm ; https://www.bible.com/bible/817/ et ceci https://www.livelingua.com/peace-corps/Fon/Guide%20d%27apprentissage%20en%20Fon.pdf pour me Traduire ce texte en Fon : {prompt}"
							}
						]
				)
		return response.choices[0].message.content
	except Execption as e :
		return f"Error : {e}. Please ensure you are using valid credentials."


In [ ]:
from googletrans import Translator

In [ ]:
# FONCTION DE TRADUCTION AVEC GOOGLE TRANSLATE GRATUIT

# Initialiser le traducteur
translator = Translator()

# Fonction de traduction avec Google Translate
def call_google_translate(prompt):
    if prompt.strip() == "":
        return ""
    try:
        result = translator.translate(prompt, src="fr", dest="fon")  # 'fon' est le code pour le fon si reconnu, sinon 'fon' n'est pas toujours supporté nativement
        return result.text
    except Exception as e:
        return f"Error: {e}. Please ensure you have internet connection."



In [ ]:
# Test
prompt="il fait nuit,  allumer les lampes. et a demain"
response_test_openai = call_openai_api(prompt)
response_test_google_g = call_google_translate(prompt)

print(response_test_openai)
print(response_test_google_g)

En langue fon, voici la traduction du texte :

**"À gbé, tò tàn azi. À wézin."**  

Cela signifie : "Il fait nuit, allumer les lampes. Et à demain."


In [ ]:
#
traductions = []

In [ ]:
# Traduction ligne par ligne
for index, row in df.iterrows():
    contenu = row["Contenu"]
    print(f"Traduction de l'article {index + 1}/{len(df)} : {row['Nom de l\'Article']}")
    traduction = call_openai_api(contenu)
    traductions.append(traduction)
    time.sleep(1)  # Pour éviter de dépasser le quota

df["traduction"] = traductions

# Sauvegarde dans un nouveau fichier CSV
df.to_csv("/content/drive/MyDrive/wikipedia_articles_fr_contents_from_1_to_15359_traduits.csv", index=False)